# 🧠 TRM Architecture Study (H/L Cycles)

One-click runnable notebook to compare **different TRM architectures**, focusing on **`H_cycles` (H)** and **`L_cycles` (L)**, while keeping the overall training loop and compute budget consistent.

## ✅ What changed vs the previous ACT notebook?
- **Before**: compared different `halt_max_steps` (ACT step cap).
- **Now**: compare different **(H, L)** settings (architecture), with **ACT early stopping disabled** during training for a fair ablation.

## ✅ Alignment with Trelis Implementation
This notebook is still structurally aligned with `TinyRecursiveModels-Trelis` (notably `config/arch/trm.yaml` + `cfg_pretrain*.yaml`) for the baseline hyperparameters (except where explicitly overridden by the sweep).

**Intentional differences**
- Single GPU (Trelis uses multi-GPU)
- AdamW optimizer (Trelis uses Muon)


In [ ]:
# ============================================================================
# Cell 1: Configuration - EDIT THIS
# ============================================================================
import os

# API Keys
WANDB_API_KEY = '' # Or set directly
HF_TOKEN = ""

# --- Architecture Experiment Selection ---
# You can either run a single config (SELECTED_CONFIG) or a sweep in Cell 13.

# 💡 Recommended 5-Config Set for Initial Comparison:
#   1. H2-L2  (minimal, efficiency baseline - 12 blocks/step)
#   2. H3-L4  (baseline, standard - 30 blocks/step) ⭐
#   3. H4-L2  (high H, low L - different ratio - 24 blocks/step)
#   4. H3-L6  (baseline H, more L - 42 blocks/step)
#   5. H4-L4  (balanced high compute - 40 blocks/step)
#
SELECTED_CONFIG = 'H3-L4'  # ⭐ Change this to run different configs

# --- Training Settings (step-based, aligned with Trelis training semantics) ---
MAX_TRAIN_STEPS = 5000
BATCH_SIZE = 2048
EVAL_EVERY_STEPS = max(1, MAX_TRAIN_STEPS // 5)
MIN_EVAL_INTERVAL = 0

# --- Compute Budget (carry-based ACT; disable q-based early halt) ---
HALT_MAX_STEPS = 16

# --- Dataset Settings ---
TRAIN_SUBSAMPLE = 1000
TEST_SUBSAMPLE = 50_000
NUM_AUGMENT = 1000
FORCE_REBUILD = False
TEST_SEED = 42

# Compute stats helper (for reference)
L_LAYERS = 2  # Fixed in model config
def get_blocks_per_step(H, L):
    """Calculate blocks per step: H_cycles * (L_cycles + 1) * L_layers"""
    return H * (L + 1) * L_LAYERS

print(f'Selected: {SELECTED_CONFIG}, MAX_TRAIN_STEPS={MAX_TRAIN_STEPS}, BatchSize: {BATCH_SIZE}, HALT_MAX_STEPS={HALT_MAX_STEPS}')
print(f'Eval every {EVAL_EVERY_STEPS} steps, TEST_SUBSAMPLE={TEST_SUBSAMPLE:,}, TRAIN_SUBSAMPLE={TRAIN_SUBSAMPLE:,}, NUM_AUGMENT={NUM_AUGMENT:,}')

# Print selected config compute stats
if '-' in SELECTED_CONFIG:
    try:
        H, L = map(int, SELECTED_CONFIG.replace('H', '').replace('L', '').split('-'))
        blocks_per_step = get_blocks_per_step(H, L)
        blocks_per_puzzle = blocks_per_step * HALT_MAX_STEPS
        print(f'📊 {SELECTED_CONFIG} Compute: {blocks_per_step} blocks/step, {blocks_per_puzzle} blocks/puzzle')
    except:
        pass

In [ ]:
# ============================================================================
# Cell 2: Install Dependencies
# ============================================================================
!pip install -q torch einops tqdm numpy pydantic wandb coolname datasets huggingface_hub pandas matplotlib
print('Dependencies installed')

In [ ]:
# ============================================================================
# Cell 3: Imports & GPU Setup
# ============================================================================
from typing import Optional, Any, Sequence, List, Tuple, Dict
from dataclasses import dataclass
import os, math, json, shutil, copy, time
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, IterableDataset
import numpy as np
from tqdm import tqdm
import wandb
import coolname
import pydantic
from pydantic import BaseModel
import einops
from torch.nn.functional import scaled_dot_product_attention

IGNORE_LABEL_ID = -100

if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    print(f'GPU: {gpu}')
    if 'H100' in gpu or 'A100' in gpu:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        print('TF32 enabled')
    torch.cuda.set_device(0)

if WANDB_API_KEY:
    wandb.login(key=WANDB_API_KEY)
    print('Wandb logged in')
else:
    os.environ['WANDB_DISABLED'] = 'true'
    print('WANDB_API_KEY not set; wandb disabled')

In [ ]:
# ============================================================================
# Cell 4: Common Utilities
# ============================================================================
def trunc_normal_init_(tensor, std=1.0, lower=-2.0, upper=2.0):
    with torch.no_grad():
        if std == 0:
            tensor.zero_()
        else:
            sqrt2 = math.sqrt(2)
            a, b = math.erf(lower/sqrt2), math.erf(upper/sqrt2)
            z = (b-a)/2
            c = (2*math.pi)**-0.5
            pdf_u, pdf_l = c*math.exp(-0.5*lower**2), c*math.exp(-0.5*upper**2)
            comp_std = std/math.sqrt(1-(upper*pdf_u-lower*pdf_l)/z-((pdf_u-pdf_l)/z)**2)
            tensor.uniform_(a, b).erfinv_().mul_(sqrt2*comp_std).clip_(lower*comp_std, upper*comp_std)
    return tensor

CosSin = Tuple[torch.Tensor, torch.Tensor]
def _find_multiple(a, b): return (-(a//-b))*b
def rotate_half(x): return torch.cat((-x[..., x.shape[-1]//2:], x[..., :x.shape[-1]//2]), dim=-1)
def apply_rotary_pos_emb(q, k, cos, sin):
    orig = q.dtype; q, k = q.to(cos.dtype), k.to(cos.dtype)
    return ((q*cos.unsqueeze(-2))+(rotate_half(q)*sin.unsqueeze(-2))).to(orig), ((k*cos.unsqueeze(-2))+(rotate_half(k)*sin.unsqueeze(-2))).to(orig)
def rms_norm(x, eps):
    dt = x.dtype; x = x.float(); return (x*torch.rsqrt(x.square().mean(-1,keepdim=True)+eps)).to(dt)
print('Utilities loaded')

In [ ]:
# ============================================================================
# Cell 5: Layers with LoRA Support
# ============================================================================
class CastedLinear(nn.Module):
    def __init__(self, in_f, out_f, bias):
        super().__init__()
        self.weight = nn.Parameter(trunc_normal_init_(torch.empty((out_f, in_f)), std=1.0/(in_f**0.5)))
        self.bias = nn.Parameter(torch.zeros(out_f)) if bias else None
        self._lora_rank = 0; self._lora_alpha = 1.0; self._lora_scaling = 1.0
        self._lora_dropout = None; self._lora_A = None; self._lora_B = None
    def forward(self, x):
        out = F.linear(x, self.weight.to(x.dtype), self.bias.to(x.dtype) if self.bias is not None else None)
        if self._lora_rank > 0 and self._lora_A is not None:
            lx = self._lora_dropout(x) if self._lora_dropout else x
            out = out + F.linear(F.linear(lx.to(self._lora_A.dtype), self._lora_A), self._lora_B).to(out.dtype) * self._lora_scaling
        return out
    def enable_lora(self, rank, alpha=None, dropout=0.0, train_base=False, train_bias=False):
        if rank <= 0: return
        if self._lora_rank > 0: raise RuntimeError('LoRA already enabled')
        self._lora_rank = rank
        self._lora_alpha = float(alpha if alpha else rank)
        self._lora_scaling = self._lora_alpha / rank
        self._lora_dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        self._lora_A = nn.Parameter(torch.zeros(rank, self.weight.shape[1]))
        self._lora_B = nn.Parameter(torch.zeros(self.weight.shape[0], rank))
        nn.init.kaiming_uniform_(self._lora_A, a=math.sqrt(5)); nn.init.zeros_(self._lora_B)
        if not train_base: self.weight.requires_grad = False
        if self.bias is not None and not train_bias: self.bias.requires_grad = False

class CastedEmbedding(nn.Module):
    def __init__(self, n, d, std, dtype):
        super().__init__()
        self.cast_to = dtype
        self.embedding_weight = nn.Parameter(trunc_normal_init_(torch.empty(n, d), std=std))
    def forward(self, x): return F.embedding(x, self.embedding_weight.to(self.cast_to))

class RotaryEmbedding(nn.Module):
    def __init__(self, dim, max_pos, base, device=None):
        super().__init__()
        inv_freq = 1.0/(base**(torch.arange(0, dim, 2, dtype=torch.float32, device=device)/dim))
        freqs = torch.outer(torch.arange(max_pos, dtype=torch.float32, device=device), inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.cos_cached = nn.Buffer(emb.cos(), persistent=False)
        self.sin_cached = nn.Buffer(emb.sin(), persistent=False)
    def forward(self): return self.cos_cached, self.sin_cached

class Attention(nn.Module):
    def __init__(self, hidden, head_dim, heads, kv_heads, causal=False):
        super().__init__()
        self.hidden, self.head_dim, self.heads, self.kv_heads, self.causal = hidden, head_dim, heads, kv_heads, causal
        self.out_size = head_dim * heads
        self.qkv_proj = CastedLinear(hidden, (heads + 2*kv_heads)*head_dim, False)
        self.o_proj = CastedLinear(self.out_size, hidden, False)
    def forward(self, cos_sin, x):
        B, L, _ = x.shape
        qkv = self.qkv_proj(x).view(B, L, self.heads + 2*self.kv_heads, self.head_dim)
        q, k, v = qkv[:,:,:self.heads], qkv[:,:,self.heads:self.heads+self.kv_heads], qkv[:,:,self.heads+self.kv_heads:]
        if cos_sin: q, k = apply_rotary_pos_emb(q, k, *cos_sin)
        q, k, v = (einops.rearrange(t, 'B S H D -> B H S D') for t in (q, k, v))
        out = scaled_dot_product_attention(q, k, v, is_causal=self.causal)
        return self.o_proj(einops.rearrange(out, 'B H S D -> B S H D').reshape(B, L, self.out_size))

class SwiGLU(nn.Module):
    def __init__(self, hidden, expansion):
        super().__init__()
        inter = _find_multiple(round(expansion*hidden*2/3), 256)
        self.gate_up = CastedLinear(hidden, inter*2, False)
        self.down = CastedLinear(inter, hidden, False)
    def forward(self, x):
        g, u = self.gate_up(x).chunk(2, dim=-1)
        return self.down(F.silu(g) * u)

def enable_lora_for_model(model, rank, alpha=None, dropout=0.0, train_base=False, train_bias=False):
    if rank <= 0: return 0
    cnt = 0
    for m in model.modules():
        if isinstance(m, CastedLinear):
            m.enable_lora(rank, alpha, dropout, train_base, train_bias); cnt += 1
    return cnt

def count_parameters(model):
    total = trainable = lora = 0
    for n, p in model.named_parameters():
        total += p.numel()
        if p.requires_grad: trainable += p.numel()
        if '_lora_' in n: lora += p.numel()
    return {'total': total, 'trainable': trainable, 'lora': lora, 'ratio': trainable/total if total else 0}

print('Layers loaded')

In [ ]:
# ============================================================================
# Cell 6: TRM Model (Aligned with TinyRecursiveModels-Trelis)
# ============================================================================

# NOTE:
# - This model matches Trelis TRM semantics for puzzle embeddings (CastedSparseEmbedding) and carry-based ACT.
# - We add a small switch `disable_act` to disable q-logits early halting during training.

@dataclass
class TRMInnerCarry:
    z_H: torch.Tensor
    z_L: torch.Tensor

@dataclass
class TRMCarry:
    inner: TRMInnerCarry
    steps: torch.Tensor
    halted: torch.Tensor
    data: Dict[str, torch.Tensor]


class CastedSparseEmbedding(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int, batch_size: int, init_std: float, cast_to: torch.dtype):
        super().__init__()
        self.cast_to = cast_to

        self.weights = nn.Buffer(trunc_normal_init_(torch.empty((num_embeddings, embedding_dim)), std=init_std), persistent=True)
        self.local_weights = nn.Buffer(torch.zeros(batch_size, embedding_dim, requires_grad=True), persistent=False)
        self.local_ids = nn.Buffer(torch.zeros(batch_size, dtype=torch.int32), persistent=False)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        if not self.training:
            return self.weights[inputs].to(self.cast_to)
        with torch.no_grad():
            self.local_weights.copy_(self.weights[inputs])
            self.local_ids.copy_(inputs)
        return self.local_weights.to(self.cast_to)


class CastedSparseEmbeddingSignSGD(torch.optim.Optimizer):
    def __init__(self, params, lr: float = 1e-3, weight_decay: float = 1e-2):
        defaults = dict(lr=lr, weight_decay=weight_decay)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        for group in self.param_groups:
            local_weights_grad = None
            local_ids = None
            weights = None
            for p in group['params']:
                if getattr(p, 'requires_grad', False):
                    local_weights_grad = p.grad
                elif p.ndim == 1:
                    local_ids = p
                elif p.ndim == 2:
                    weights = p
            if local_weights_grad is None or local_ids is None or weights is None:
                continue

            # Unique ids for this batch
            grad_ids, inv = local_ids.unique(return_inverse=True)
            grad = torch.zeros((grad_ids.shape[0], local_weights_grad.shape[1]), dtype=local_weights_grad.dtype, device=local_weights_grad.device)
            grad.scatter_add_(0, inv.unsqueeze(-1).expand(-1, grad.shape[1]), local_weights_grad)

            lr = float(group['lr'])
            wd = float(group['weight_decay'])
            p = weights[grad_ids]
            p.mul_(1.0 - lr * wd).add_(torch.sign(grad), alpha=-lr)
            weights[grad_ids] = p


class TRMConfig(BaseModel):
    batch_size: int
    seq_len: int
    vocab_size: int
    num_puzzle_identifiers: int

    H_cycles: int
    L_cycles: int
    H_layers: int
    L_layers: int

    hidden_size: int
    expansion: float
    num_heads: int
    pos_encodings: str

    rms_norm_eps: float = 1e-5
    rope_theta: float = 10000.0

    halt_max_steps: int
    halt_exploration_prob: float
    halt_max_steps_eval: Optional[int] = None

    forward_dtype: str = 'bfloat16'

    puzzle_emb_ndim: int = 0
    puzzle_emb_len: int = 1

    mlp_t: bool = False
    no_ACT_continue: bool = True

    puzzle_emb_dropout: float = 0.0
    grid_token_dropout: float = 0.0

    # Extra switch (not in Trelis): disable q-logits early halting during training
    disable_act: bool = False


class TRMBlock(nn.Module):
    def __init__(self, cfg: TRMConfig):
        super().__init__()
        self.cfg = cfg
        if cfg.mlp_t:
            self.mlp_t = SwiGLU(cfg.seq_len + cfg.puzzle_emb_len, cfg.expansion)
        else:
            self.attn = Attention(cfg.hidden_size, cfg.hidden_size // cfg.num_heads, cfg.num_heads, cfg.num_heads, False)
        self.mlp = SwiGLU(cfg.hidden_size, cfg.expansion)
        self.eps = cfg.rms_norm_eps

    def forward(self, cos_sin, x):
        if self.cfg.mlp_t:
            x = x.transpose(1, 2)
            x = rms_norm(x + self.mlp_t(x), self.eps)
            x = x.transpose(1, 2)
        else:
            x = rms_norm(x + self.attn(cos_sin, x), self.eps)
        return rms_norm(x + self.mlp(x), self.eps)


class TRMReasoning(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)

    def forward(self, x, inj, **kw):
        x = x + inj
        for l in self.layers:
            x = l(kw.get('cos_sin'), x)
        return x


class TRMInner(nn.Module):
    def __init__(self, cfg: TRMConfig):
        super().__init__()
        self.cfg = cfg
        self.dtype = getattr(torch, cfg.forward_dtype)

        self.embed_scale = math.sqrt(cfg.hidden_size)
        embed_init_std = 1.0 / self.embed_scale

        self.embed_tokens = CastedEmbedding(cfg.vocab_size, cfg.hidden_size, embed_init_std, self.dtype)
        self.lm_head = CastedLinear(cfg.hidden_size, cfg.vocab_size, False)
        self.q_head = CastedLinear(cfg.hidden_size, 2, True)

        # puzzle embedding
        self.puzzle_emb_len = (-(cfg.puzzle_emb_ndim // -cfg.hidden_size)) if (cfg.puzzle_emb_len == 0) else cfg.puzzle_emb_len
        if cfg.puzzle_emb_ndim > 0:
            self.puzzle_emb = CastedSparseEmbedding(cfg.num_puzzle_identifiers, cfg.puzzle_emb_ndim, batch_size=cfg.batch_size, init_std=0.0, cast_to=self.dtype)

        if cfg.pos_encodings == 'rope':
            self.rotary = RotaryEmbedding(cfg.hidden_size // cfg.num_heads, cfg.seq_len + self.puzzle_emb_len, cfg.rope_theta)
        elif cfg.pos_encodings == 'learned':
            self.pos_emb = CastedEmbedding(cfg.seq_len + self.puzzle_emb_len, cfg.hidden_size, embed_init_std, self.dtype)

        self.L_level = TRMReasoning([TRMBlock(cfg) for _ in range(cfg.L_layers)])

        self.H_init = nn.Buffer(trunc_normal_init_(torch.empty(cfg.hidden_size, dtype=self.dtype), std=1), persistent=True)
        self.L_init = nn.Buffer(trunc_normal_init_(torch.empty(cfg.hidden_size, dtype=self.dtype), std=1), persistent=True)

        with torch.no_grad():
            self.q_head.weight.zero_()
            self.q_head.bias.fill_(-5)

    def _input_embeddings(self, inputs: torch.Tensor, puzzle_identifiers: torch.Tensor):
        emb = self.embed_tokens(inputs.to(torch.int32))

        if self.cfg.puzzle_emb_ndim > 0:
            pe = self.puzzle_emb(puzzle_identifiers)
            pad_count = self.puzzle_emb_len * self.cfg.hidden_size - pe.shape[-1]
            if pad_count > 0:
                pe = F.pad(pe, (0, pad_count))
            emb = torch.cat((pe.view(-1, self.puzzle_emb_len, self.cfg.hidden_size), emb), dim=-2)

        if self.cfg.pos_encodings == 'learned':
            emb = 0.707106781 * (emb + self.pos_emb.embedding_weight.to(self.dtype))

        return self.embed_scale * emb

    def empty_carry(self, batch_size: int):
        return TRMInnerCarry(
            z_H=torch.empty(batch_size, self.cfg.seq_len + self.puzzle_emb_len, self.cfg.hidden_size, dtype=self.dtype),
            z_L=torch.empty(batch_size, self.cfg.seq_len + self.puzzle_emb_len, self.cfg.hidden_size, dtype=self.dtype),
        )

    def reset_carry(self, reset_flag: torch.Tensor, carry: TRMInnerCarry):
        return TRMInnerCarry(
            z_H=torch.where(reset_flag.view(-1, 1, 1), self.H_init, carry.z_H),
            z_L=torch.where(reset_flag.view(-1, 1, 1), self.L_init, carry.z_L),
        )

    def forward(self, carry: TRMInnerCarry, batch: Dict[str, torch.Tensor]):
        cs = self.rotary() if hasattr(self, 'rotary') else None

        input_embeddings = self._input_embeddings(batch['inputs'], batch['puzzle_identifiers'])

        z_H, z_L = carry.z_H, carry.z_L

        with torch.no_grad():
            for _ in range(self.cfg.H_cycles - 1):
                for _ in range(self.cfg.L_cycles):
                    z_L = self.L_level(z_L, z_H + input_embeddings, cos_sin=cs)
                z_H = self.L_level(z_H, z_L, cos_sin=cs)

        for _ in range(self.cfg.L_cycles):
            z_L = self.L_level(z_L, z_H + input_embeddings, cos_sin=cs)
        z_H = self.L_level(z_H, z_L, cos_sin=cs)

        new_carry = TRMInnerCarry(z_H=z_H.detach(), z_L=z_L.detach())
        logits = self.lm_head(z_H)[:, self.puzzle_emb_len:]
        q_logits = self.q_head(z_H[:, 0]).to(torch.float32)

        return new_carry, {'logits': logits, 'q_halt_logits': q_logits[:, 0], 'q_continue_logits': q_logits[:, 1]}


class TinyRecursiveReasoningModel_ACTV1(nn.Module):
    def __init__(self, cfg_dict: dict):
        super().__init__()
        self.config = TRMConfig(**cfg_dict)
        self.inner = TRMInner(self.config)

    def initial_carry(self, batch: Dict[str, torch.Tensor]):
        B = batch['inputs'].shape[0]
        return TRMCarry(
            inner=self.inner.empty_carry(B),
            steps=torch.zeros(B, dtype=torch.int32, device=batch['inputs'].device),
            halted=torch.ones(B, dtype=torch.bool, device=batch['inputs'].device),
            data={k: torch.empty_like(v) for k, v in batch.items()},
        )

    def forward(self, carry: TRMCarry, batch: Dict[str, torch.Tensor], **kw):
        new_inner = self.inner.reset_carry(carry.halted, carry.inner)
        new_steps = torch.where(carry.halted, torch.zeros_like(carry.steps), carry.steps)
        new_data = {k: torch.where(carry.halted.view((-1,) + (1,) * (v.ndim - 1)), batch[k], v) for k, v in carry.data.items()}

        new_inner, out = self.inner(new_inner, new_data)

        with torch.no_grad():
            new_steps = new_steps + 1
            halt_limit = self.config.halt_max_steps if (self.training or self.config.halt_max_steps_eval is None) else self.config.halt_max_steps_eval
            is_last_step = new_steps >= halt_limit
            halted = is_last_step

            # Disable q-logits early halting if requested.
            if (not self.config.disable_act) and self.training and (self.config.halt_max_steps > 1):
                if self.config.no_ACT_continue:
                    halted = halted | (out['q_halt_logits'] > 0)
                else:
                    halted = halted | (out['q_halt_logits'] > out['q_continue_logits'])

                min_halt_steps = (torch.rand_like(out['q_halt_logits']) < self.config.halt_exploration_prob) * torch.randint_like(
                    new_steps, low=2, high=self.config.halt_max_steps + 1
                )
                halted = halted & (new_steps >= min_halt_steps)

        return TRMCarry(new_inner, new_steps, halted, new_data), out


print('TRM Model loaded (aligned with Trelis puzzle embeddings + carry)')

In [ ]:
# ============================================================================
# Cell 7: Loss Head (Aligned with Trelis ACTLossHead)
# ============================================================================

def s(x, eps=1e-30):
    return torch.where(x < 0, 1 / (1 - x + eps), x + 1)


def log_stablemax(x, dim=-1):
    sx = s(x)
    return torch.log(sx / sx.sum(dim=dim, keepdim=True))


def stablemax_ce(logits, labels, ignore_index=-100, valid_mask=None):
    lp = log_stablemax(logits.double(), -1)
    if valid_mask is None:
        valid_mask = labels != ignore_index
    tl = torch.where(valid_mask, labels, 0)
    plp = torch.gather(lp, index=tl.long().unsqueeze(-1), dim=-1).squeeze(-1)
    return -torch.where(valid_mask, plp, 0.0)


class TRMLossHead(nn.Module):
    """Trelis ACTLossHead-equivalent, but bound to our local model wrapper."""

    def __init__(self, model: nn.Module, loss_type: str):
        super().__init__()
        self.model = model
        self.loss_fn = stablemax_ce

    def initial_carry(self, *args, **kwargs):
        return self.model.initial_carry(*args, **kwargs)

    def forward(self, return_keys: Sequence[str], **model_kwargs):
        new_carry, outputs = self.model(**model_kwargs)
        labels = new_carry.data['labels']

        with torch.no_grad():
            outputs['preds'] = torch.argmax(outputs['logits'], dim=-1)

            mask = labels != IGNORE_LABEL_ID
            loss_counts = mask.sum(-1)
            loss_divisor = loss_counts.clamp_min(1).unsqueeze(-1)

            is_correct = mask & (outputs['preds'] == labels)
            seq_is_correct = is_correct.sum(-1) == loss_counts

            valid_metrics = new_carry.halted & (loss_counts > 0)
            metrics = {
                'count': valid_metrics.sum(),
                'accuracy': torch.where(valid_metrics, (is_correct.float() / loss_divisor).sum(-1), 0.0).sum(),
                'exact_accuracy': (valid_metrics & seq_is_correct).sum(),
                'q_halt_accuracy': (valid_metrics & ((outputs['q_halt_logits'] >= 0) == seq_is_correct)).sum(),
                'steps': torch.where(valid_metrics, new_carry.steps, 0).sum(),
            }

        lm_loss = (self.loss_fn(outputs['logits'], labels, ignore_index=IGNORE_LABEL_ID, valid_mask=mask) / loss_divisor).sum()
        q_halt_loss = F.binary_cross_entropy_with_logits(outputs['q_halt_logits'], seq_is_correct.to(outputs['q_halt_logits'].dtype), reduction='sum')

        metrics.update({'lm_loss': lm_loss.detach(), 'q_halt_loss': q_halt_loss.detach()})

        q_continue_loss = 0
        if 'target_q_continue' in outputs:
            q_continue_loss = F.binary_cross_entropy_with_logits(outputs['q_continue_logits'], outputs['target_q_continue'], reduction='sum')
            metrics['q_continue_loss'] = q_continue_loss.detach()

        detached_outputs = {k: outputs[k].detach() for k in return_keys if k in outputs}

        total_loss = lm_loss + 0.5 * (q_halt_loss + q_continue_loss)
        return new_carry, total_loss, metrics, detached_outputs, new_carry.halted.all()


print('Loss Head loaded (aligned with Trelis ACTLossHead)')

In [ ]:
# ============================================================================
# Cell 8: Dataset Classes (Aligned with TinyRecursiveModels-Trelis puzzle_dataset.py)
# ============================================================================
from torch.utils.data import get_worker_info


def _sample_batch(rng, group_order, puzzle_indices, group_indices, start_index, global_batch_size):
    batch = []
    batch_puzzle_indices = []
    current_size = 0

    while (start_index < group_order.size) and (current_size < global_batch_size):
        group_id = group_order[start_index]
        puzzle_id = rng.integers(group_indices[group_id], group_indices[group_id + 1])
        start_index += 1

        puzzle_start = puzzle_indices[puzzle_id]
        puzzle_size = int(puzzle_indices[puzzle_id + 1] - puzzle_start)
        if puzzle_size <= 0:
            continue

        append_size = min(puzzle_size, global_batch_size - current_size)
        batch_puzzle_indices.append(np.full(append_size, puzzle_id, dtype=np.int32))

        if append_size == puzzle_size:
            choices = rng.permutation(puzzle_size)
        else:
            choices = rng.choice(puzzle_size, append_size, replace=False)
        batch.append(puzzle_start + choices)
        current_size += append_size

    if len(batch) == 0:
        return start_index, np.empty((0,), dtype=np.int64), np.empty((0,), dtype=np.int32)

    return start_index, np.concatenate(batch), np.concatenate(batch_puzzle_indices)


class PuzzleDatasetMetadata(pydantic.BaseModel):
    pad_id: int
    ignore_label_id: Optional[int]
    blank_identifier_id: int
    vocab_size: int
    seq_len: int
    num_puzzle_identifiers: int
    total_groups: int
    mean_puzzle_examples: float
    total_puzzles: int
    sets: List[str]


class PuzzleDatasetConfig(pydantic.BaseModel):
    seed: int
    dataset_paths: List[str]
    global_batch_size: int
    test_set_mode: bool
    epochs_per_iter: int
    rank: int = 0
    num_replicas: int = 1


class PuzzleDataset(IterableDataset):
    def __init__(self, cfg: PuzzleDatasetConfig, split: str = 'train'):
        super().__init__()
        self.cfg = cfg
        self.split = split

        # Merge metadata across dataset_paths
        prev = None
        mean_puzzle_examples = 0.0
        total_puzzles = 0
        total_groups = 0
        num_identifiers = 0

        for dataset_path in cfg.dataset_paths:
            with open(os.path.join(dataset_path, split, 'dataset.json')) as f:
                md = PuzzleDatasetMetadata(**json.load(f))

            if prev is None:
                prev = md
            else:
                assert prev.seq_len == md.seq_len
                assert prev.vocab_size == md.vocab_size
                assert prev.pad_id == md.pad_id
                assert prev.ignore_label_id == md.ignore_label_id
                assert prev.blank_identifier_id == md.blank_identifier_id
                assert prev.sets == md.sets

            mean_puzzle_examples += float(md.mean_puzzle_examples) * int(md.total_puzzles)
            total_puzzles += int(md.total_puzzles)
            total_groups += int(md.total_groups)
            num_identifiers += int(md.num_puzzle_identifiers)

        assert prev is not None
        mean_puzzle_examples = mean_puzzle_examples / max(1, total_puzzles)

        self.metadata = PuzzleDatasetMetadata(
            pad_id=prev.pad_id,
            ignore_label_id=prev.ignore_label_id,
            blank_identifier_id=prev.blank_identifier_id,
            vocab_size=prev.vocab_size,
            seq_len=prev.seq_len,
            num_puzzle_identifiers=num_identifiers,
            total_groups=total_groups,
            mean_puzzle_examples=mean_puzzle_examples,
            total_puzzles=total_puzzles,
            sets=prev.sets,
        )

        assert cfg.global_batch_size % cfg.num_replicas == 0
        self.local_bs = cfg.global_batch_size // cfg.num_replicas

        self._data = None
        self._iters = 0

    def _lazy_load_dataset(self):
        if self._data is not None:
            return

        mmap_modes = {
            'inputs': 'r',
            'labels': 'r',
            'puzzle_identifiers': None,
            'puzzle_indices': None,
            'group_indices': None,
        }

        self._data = {}
        for set_name in self.metadata.sets:
            for i, dataset_path in enumerate(self.cfg.dataset_paths):
                set_name_ = set_name + (str(i) if i > 0 else '')
                dataset = {
                    k: np.load(os.path.join(dataset_path, self.split, f'{set_name}__{k}.npy'), mmap_mode=mmap_modes[k])
                    for k in mmap_modes
                }

                group_indices = dataset['group_indices']
                num_groups = group_indices.size - 1
                puzzle_group_ids = np.empty(int(group_indices[-1]), dtype=np.int32)
                for gid in range(num_groups):
                    start = int(group_indices[gid])
                    end = int(group_indices[gid + 1])
                    puzzle_group_ids[start:end] = gid
                dataset['puzzle_group_ids'] = puzzle_group_ids

                self._data[set_name_] = dataset

    def _collate_batch(self, batch: Dict[str, np.ndarray]):
        batch = {k: v.astype(np.int32) for k, v in batch.items()}

        if self.metadata.ignore_label_id is not None:
            batch['labels'][batch['labels'] == self.metadata.ignore_label_id] = IGNORE_LABEL_ID

        if batch['puzzle_identifiers'].size < self.local_bs:
            pad = self.local_bs - batch['puzzle_identifiers'].size
            pad_values = {
                'inputs': self.metadata.pad_id,
                'labels': IGNORE_LABEL_ID,
                'puzzle_identifiers': self.metadata.blank_identifier_id,
                'task_identifiers': -1,
            }
            batch = {
                k: np.pad(v, ((0, pad),) + ((0, 0),) * (v.ndim - 1), constant_values=pad_values[k])
                for k, v in batch.items()
            }

        return {k: torch.from_numpy(v) for k, v in batch.items()}

    def _iter_test(self):
        for set_name, dataset in self._data.items():
            total_examples = len(dataset['inputs'])
            start_index = 0
            while start_index < total_examples:
                end_index = min(total_examples, start_index + self.cfg.global_batch_size)

                local_start = start_index + self.cfg.rank * self.local_bs
                local_end = min(start_index + (self.cfg.rank + 1) * self.local_bs, end_index)

                puzzle_indices = []
                puzzle_index = np.searchsorted(dataset['puzzle_indices'], local_start, side='right') - 1
                for i in range(local_start, local_end):
                    while puzzle_index + 1 < len(dataset['puzzle_indices']) and i >= dataset['puzzle_indices'][puzzle_index + 1]:
                        puzzle_index += 1
                    puzzle_indices.append(puzzle_index)

                batch = self._collate_batch({
                    'inputs': dataset['inputs'][local_start:local_end],
                    'labels': dataset['labels'][local_start:local_end],
                    'puzzle_identifiers': dataset['puzzle_identifiers'][puzzle_indices],
                    'task_identifiers': dataset['puzzle_group_ids'][puzzle_indices],
                })

                yield set_name, batch, end_index - start_index
                start_index += self.cfg.global_batch_size

    def _iter_train(self):
        self._iters += 1
        epoch_seed = self.cfg.seed + self._iters

        rng = np.random.Generator(np.random.Philox(seed=epoch_seed))

        for set_name, dataset in self._data.items():
            group_order = np.concatenate([
                rng.permutation(dataset['group_indices'].size - 1)
                for _ in range(self.cfg.epochs_per_iter)
            ])
            start_index = 0

            while start_index < group_order.size:
                start_index, batch_indices, batch_puzzle_indices = _sample_batch(
                    rng,
                    group_order=group_order,
                    puzzle_indices=dataset['puzzle_indices'],
                    group_indices=dataset['group_indices'],
                    start_index=start_index,
                    global_batch_size=self.cfg.global_batch_size,
                )

                gbs = batch_puzzle_indices.size
                if gbs < self.cfg.global_batch_size:
                    break

                batch_indices = batch_indices[self.cfg.rank * self.local_bs:(self.cfg.rank + 1) * self.local_bs]
                batch_puzzle_indices = batch_puzzle_indices[self.cfg.rank * self.local_bs:(self.cfg.rank + 1) * self.local_bs]

                batch = self._collate_batch({
                    'inputs': dataset['inputs'][batch_indices],
                    'labels': dataset['labels'][batch_indices],
                    'puzzle_identifiers': dataset['puzzle_identifiers'][batch_puzzle_indices],
                    'task_identifiers': dataset['puzzle_group_ids'][batch_puzzle_indices],
                })

                yield set_name, batch, gbs

    def __iter__(self):
        worker_info = get_worker_info()
        assert worker_info is None or worker_info.num_workers == 1

        self._lazy_load_dataset()

        if self.cfg.test_set_mode:
            yield from self._iter_test()
        else:
            yield from self._iter_train()


print('Dataset loaded (aligned with Trelis)')

In [ ]:
# ============================================================================
# Cell 9: Training Framework (Single GPU + AdamW + Sparse Embedding SignSGD)
# ============================================================================
class LossConfig(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(extra='allow')
    name: str


class ArchConfig(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(extra='allow')
    name: str
    loss: LossConfig


class PretrainConfig(pydantic.BaseModel):
    """Minimal pretrain config aligned with Trelis training semantics, but step-based."""

    arch: ArchConfig
    data_paths: List[str]
    data_paths_test: List[str] = []

    global_batch_size: int

    # Step-based control
    max_steps: int
    eval_every_steps: int
    min_eval_interval: int = 0

    # Optimizer hyperparams (trunk)
    lr: float
    lr_min_ratio: float
    lr_warmup_steps: int
    weight_decay: float
    beta1: float
    beta2: float

    # Puzzle embedding training (aligned to Trelis defaults)
    puzzle_emb_lr: float = 1e-2
    puzzle_emb_weight_decay: float = 0.1

    # Names / logging
    project_name: Optional[str] = None
    run_name: Optional[str] = None
    checkpoint_path: Optional[str] = None
    checkpoint_every_eval: bool = False

    ema: bool = False
    ema_rate: float = 0.999

    seed: int = 0


@dataclass
class TrainState:
    model: nn.Module
    optimizers: List[Any]
    optimizer_lrs: List[float]
    optimizer_tags: List[str]
    carry: Any
    step: int
    total_steps: int


class EMAHelper:
    def __init__(self, mu=0.999):
        self.mu = mu
        self.shadow = {}

    def register(self, m):
        for n, p in m.named_parameters():
            if p.requires_grad:
                self.shadow[n] = p.data.clone()

    def update(self, m):
        for n, p in m.named_parameters():
            if p.requires_grad and n in self.shadow:
                self.shadow[n] = self.mu * self.shadow[n] + (1 - self.mu) * p.data

    def ema_copy(self, m):
        mc = copy.deepcopy(m)
        for n, p in mc.named_parameters():
            if n in self.shadow:
                p.data.copy_(self.shadow[n])
        return mc


def cosine_lr(step, base_lr, warmup, total, min_ratio=0.0):
    if step < warmup:
        return base_lr * step / max(1, warmup)
    prog = (step - warmup) / max(1, total - warmup)
    return base_lr * (min_ratio + (1 - min_ratio) * 0.5 * (1 + math.cos(math.pi * prog)))


def create_dataloader(cfg: PretrainConfig, split: str, epochs_per_iter: int):
    ds = PuzzleDataset(
        PuzzleDatasetConfig(
            seed=cfg.seed,
            dataset_paths=cfg.data_paths_test if split == 'test' and cfg.data_paths_test else cfg.data_paths,
            global_batch_size=cfg.global_batch_size,
            test_set_mode=(split == 'test'),
            epochs_per_iter=epochs_per_iter,
            rank=0,
            num_replicas=1,
        ),
        split,
    )
    loader = DataLoader(ds, batch_size=None, num_workers=1, prefetch_factor=8, pin_memory=True, persistent_workers=True)
    return loader, ds.metadata


def _get_sparse_puzzle_emb_buffers(loss_head_model: nn.Module):
    # loss_head_model is TRMLossHead
    core = getattr(loss_head_model, 'model', None)
    inner = getattr(core, 'inner', None) if core is not None else None
    puzzle_emb = getattr(inner, 'puzzle_emb', None) if inner is not None else None
    if puzzle_emb is None:
        return None
    return list(puzzle_emb.buffers())


def create_model(cfg: PretrainConfig, meta: PuzzleDatasetMetadata):
    model_cfg = {
        **cfg.arch.__pydantic_extra__,
        'batch_size': cfg.global_batch_size,
        'vocab_size': meta.vocab_size,
        'seq_len': meta.seq_len,
        'num_puzzle_identifiers': meta.num_puzzle_identifiers,
        'causal': False,
    }

    with torch.device('cuda'):
        m = TinyRecursiveReasoningModel_ACTV1(model_cfg)
        m = TRMLossHead(m, cfg.arch.loss.__pydantic_extra__.get('loss_type', 'stablemax_cross_entropy'))
        stats = count_parameters(m)
        print(f'Params: {stats["total"]:,} total, {stats["trainable"]:,} trainable ({stats["ratio"]:.2%})')
        if 'DISABLE_COMPILE' not in os.environ:
            m = torch.compile(m)

    optimizers: List[Any] = []
    optimizer_lrs: List[float] = []
    optimizer_tags: List[str] = []

    # Puzzle embedding optimizer (SignSGD) if present
    sparse_buffers = _get_sparse_puzzle_emb_buffers(m)
    if sparse_buffers is not None:
        optimizers.append(CastedSparseEmbeddingSignSGD(sparse_buffers, lr=cfg.puzzle_emb_lr, weight_decay=cfg.puzzle_emb_weight_decay))
        optimizer_lrs.append(cfg.puzzle_emb_lr)
        optimizer_tags.append('embedding')

    # Trunk optimizer (AdamW on trainable parameters)
    optimizers.append(
        torch.optim.AdamW(
            [p for p in m.parameters() if p.requires_grad],
            lr=cfg.lr,
            weight_decay=cfg.weight_decay,
            betas=(cfg.beta1, cfg.beta2),
        )
    )
    optimizer_lrs.append(cfg.lr)
    optimizer_tags.append('trunk')

    return m, optimizers, optimizer_lrs, optimizer_tags


def init_train_state(cfg: PretrainConfig, meta: PuzzleDatasetMetadata):
    model, opts, lrs, tags = create_model(cfg, meta)
    return TrainState(model=model, optimizers=opts, optimizer_lrs=lrs, optimizer_tags=tags, carry=None, step=0, total_steps=cfg.max_steps)


def train_batch(cfg: PretrainConfig, ts: TrainState, batch: Dict[str, torch.Tensor], gbs: int):
    step_start = time.time()
    ts.step += 1
    if ts.step > ts.total_steps:
        return None

    batch = {k: v.cuda() for k, v in batch.items()}

    if ts.carry is None:
        with torch.device('cuda'):
            ts.carry = ts.model.initial_carry(batch)

    ts.carry, loss, metrics, _, _ = ts.model(carry=ts.carry, batch=batch, return_keys=[])
    (loss / gbs).backward()

    # Step optimizers with their own LR schedules
    lr_logged = None
    for opt, base_lr, tag in zip(ts.optimizers, ts.optimizer_lrs, ts.optimizer_tags):
        lr_now = cosine_lr(ts.step, base_lr, cfg.lr_warmup_steps, ts.total_steps, cfg.lr_min_ratio if tag == 'trunk' else 0.0)
        for pg in opt.param_groups:
            pg['lr'] = lr_now
        opt.step()
        opt.zero_grad()
        if tag == 'trunk':
            lr_logged = lr_now

    step_time = time.time() - step_start

    if metrics:
        keys = sorted(metrics.keys())
        vals = torch.stack([metrics[k] for k in keys]).cpu().numpy()
        rm = {k: vals[i] for i, k in enumerate(keys)}
        cnt = max(rm.get('count', 1), 1)

        train_metrics = {f'train/{k}': v / (gbs if k.endswith('loss') else cnt) for k, v in rm.items()}
        if lr_logged is not None:
            train_metrics['train/lr'] = lr_logged
        train_metrics['train/samples_per_sec'] = gbs / step_time if step_time > 0 else 0
        return train_metrics

    return None


def evaluate(cfg: PretrainConfig, ts: TrainState, loader: DataLoader, max_batches: Optional[int] = None):
    metrics = None
    batch_count = 0
    with torch.inference_mode():
        for i, (sn, batch, gbs) in enumerate(loader):
            if max_batches is not None and i >= max_batches:
                break
            batch = {k: v.cuda() for k, v in batch.items()}
            with torch.device('cuda'):
                carry = ts.model.initial_carry(batch)

            while True:
                carry, _, m, _, done = ts.model(carry=carry, batch=batch, return_keys=[])
                if done:
                    break

            if metrics is None:
                metrics = {k: 0.0 for k in m}
            for k in m:
                metrics[k] += m[k].item() if torch.is_tensor(m[k]) else m[k]
            batch_count += 1

    if metrics:
        cnt = max(metrics.get('count', 1), 1)
        result = {f'eval/{k}': v / cnt for k, v in metrics.items() if k != 'count'}
        result['eval/batches_evaluated'] = batch_count
        return result

    return {}


print('Training framework loaded (step-based, aligned with Trelis carry + sparse puzzle emb)')

In [ ]:
# ============================================================================
# Cell 10: Architecture Configurations (H/L cycles)
# ============================================================================
# We compare different architectures by varying (H_cycles, L_cycles).
# `HALT_MAX_STEPS` (Cell 1) is kept constant and ACT early stopping is disabled.

H_CYCLES_LIST = [2, 3, 4]
L_CYCLES_LIST = [2, 4, 6]

ARCH_CONFIGS = {
    f'H{H}-L{L}': {
        'H_cycles': H,
        'L_cycles': L,
        'desc': f'Architecture: H_cycles={H}, L_cycles={L}',
    }
    for H in H_CYCLES_LIST
    for L in L_CYCLES_LIST
}

# Convenience alias for the standard Trelis baseline
if 'H3-L4' not in ARCH_CONFIGS:
    ARCH_CONFIGS['H3-L4'] = {'H_cycles': 3, 'L_cycles': 4, 'desc': 'Trelis baseline: H=3, L=4'}

print('ARCH configs:', sorted(ARCH_CONFIGS.keys()))

In [ ]:
# ============================================================================
# Cell 11: Build Sudoku Dataset (Aligned with TinyRecursiveModels-Trelis)
# ============================================================================
from datasets import load_dataset
from huggingface_hub import login

DATASET_DIR = './data/sudoku_arch'


def shuffle_sudoku(board: np.ndarray, solution: np.ndarray):
    digit_map = np.pad(np.random.permutation(np.arange(1, 10)), (1, 0))

    transpose_flag = np.random.rand() < 0.5

    bands = np.random.permutation(3)
    row_perm = np.concatenate([b * 3 + np.random.permutation(3) for b in bands])

    stacks = np.random.permutation(3)
    col_perm = np.concatenate([s * 3 + np.random.permutation(3) for s in stacks])

    mapping = np.array([row_perm[i // 9] * 9 + col_perm[i % 9] for i in range(81)])

    def apply_transformation(x: np.ndarray) -> np.ndarray:
        if transpose_flag:
            x = x.T
        new_board = x.flatten()[mapping].reshape(9, 9).copy()
        return digit_map[new_board]

    return apply_transformation(board), apply_transformation(solution)


def build_sudoku_dataset():
    train_md_path = os.path.join(DATASET_DIR, 'train', 'dataset.json')

    if os.path.exists(train_md_path) and not FORCE_REBUILD:
        try:
            with open(train_md_path) as f:
                md = json.load(f)
            expected_mean = float(1 + NUM_AUGMENT)
            if float(md.get('mean_puzzle_examples', 0.0)) >= max(1.0, expected_mean - 0.5) or NUM_AUGMENT == 0:
                print('Dataset exists, skipping build')
                return
            else:
                print('Dataset exists but has mean_puzzle_examples too small; rebuilding')
        except Exception as e:
            print(f'Dataset exists but could not read metadata ({e}); rebuilding')

    if os.path.exists(DATASET_DIR):
        shutil.rmtree(DATASET_DIR)

    if HF_TOKEN:
        os.environ['HF_TOKEN'] = HF_TOKEN
        try:
            login(token=HF_TOKEN, add_to_git_credential=False)
            print('✅ Successfully authenticated with Hugging Face Hub')
        except Exception as e:
            print(f'⚠️ Warning: Could not login to Hugging Face Hub: {e}')
    else:
        print('ℹ️ HF_TOKEN not set; will try downloading dataset without authentication')

    print('Building Sudoku dataset...')

    ds = load_dataset('sapientinc/sudoku-extreme', token=HF_TOKEN) if HF_TOKEN else load_dataset('sapientinc/sudoku-extreme')

    sample = ds['train'][0]
    puzzle_key = next((k for k in sample.keys() if k in ['question', 'puzzle', 'quiz', 'input']), None)
    solution_key = next((k for k in sample.keys() if k in ['answer', 'solution', 'output', 'target']), None)
    if puzzle_key is None or solution_key is None:
        raise ValueError(f'Cannot determine puzzle/solution columns from keys: {list(sample.keys())}')

    def convert(split: str):
        data = ds[split]
        if split == 'train':
            data = data.select(range(min(TRAIN_SUBSAMPLE, len(data))))
        elif split == 'test':
            original_size = len(data)
            if original_size > TEST_SUBSAMPLE:
                import random
                random.seed(TEST_SEED)
                np.random.seed(TEST_SEED)
                indices = list(range(original_size))
                random.shuffle(indices)
                selected_indices = sorted(indices[:TEST_SUBSAMPLE])
                data = data.select(selected_indices)
                print(f'Test set: Using {len(data):,} samples (sampled from {original_size:,}, seed={TEST_SEED})')
            else:
                print(f'Test set: Using all {len(data):,} samples')

        inputs = [np.frombuffer(str(r[puzzle_key]).replace('.', '0').encode(), dtype=np.uint8).reshape(9, 9) - ord('0') for r in data]
        labels = [np.frombuffer(str(r[solution_key]).encode(), dtype=np.uint8).reshape(9, 9) - ord('0') for r in data]

        num_augments = NUM_AUGMENT if split == 'train' else 0

        results = {k: [] for k in ['inputs', 'labels', 'puzzle_identifiers', 'puzzle_indices', 'group_indices']}
        puzzle_id = 0
        example_id = 0

        results['puzzle_indices'].append(0)
        results['group_indices'].append(0)

        for orig_inp, orig_out in zip(inputs, labels):
            for aug_idx in range(1 + num_augments):
                if aug_idx == 0:
                    inp, out = orig_inp, orig_out
                else:
                    inp, out = shuffle_sudoku(orig_inp, orig_out)

                results['inputs'].append(inp)
                results['labels'].append(out)
                example_id += 1

            puzzle_id += 1
            results['puzzle_indices'].append(example_id)
            results['puzzle_identifiers'].append(0)
            results['group_indices'].append(puzzle_id)

        def _seq_to_numpy(seq):
            arr = np.concatenate(seq).reshape(len(seq), -1)
            assert np.all((arr >= 0) & (arr <= 9))
            return (arr + 1).astype(np.int32)

        results = {
            'inputs': _seq_to_numpy(results['inputs']),
            'labels': _seq_to_numpy(results['labels']),
            'group_indices': np.array(results['group_indices'], dtype=np.int32),
            'puzzle_indices': np.array(results['puzzle_indices'], dtype=np.int32),
            'puzzle_identifiers': np.array(results['puzzle_identifiers'], dtype=np.int32),
        }

        metadata = PuzzleDatasetMetadata(
            seq_len=81,
            vocab_size=11,
            pad_id=0,
            ignore_label_id=0,
            blank_identifier_id=0,
            num_puzzle_identifiers=1,
            total_groups=len(results['group_indices']) - 1,
            mean_puzzle_examples=float(1 + num_augments),
            total_puzzles=len(results['puzzle_indices']) - 1,
            sets=['all'],
        )

        save_dir = os.path.join(DATASET_DIR, split)
        os.makedirs(save_dir, exist_ok=True)

        with open(os.path.join(save_dir, 'dataset.json'), 'w') as f:
            json.dump(metadata.model_dump(), f)

        for k, v in results.items():
            np.save(os.path.join(save_dir, f'all__{k}.npy'), v)

        print(f'{split}: inputs={results["inputs"].shape}, groups={metadata.total_groups}, puzzles={results["puzzle_indices"].size - 1}')

    convert('train')
    convert('test')

    print('Dataset built!')


build_sudoku_dataset()

In [ ]:
# ============================================================================
# Cell 12: Launch Experiment (H/L ablation, compute-normalized)
# ============================================================================
import time


def _arch_compute_stats(H_cycles: int, L_cycles: int, L_layers: int, halt_max_steps: int):
    blocks_per_step = H_cycles * (L_cycles + 1) * L_layers
    blocks_per_puzzle = halt_max_steps * blocks_per_step
    return blocks_per_step, blocks_per_puzzle


def launch_experiment(config_name: str):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    arch_cfg = ARCH_CONFIGS[config_name]
    H_cycles = arch_cfg['H_cycles']
    L_cycles = arch_cfg['L_cycles']

    print('=' * 70)
    print(f'Experiment: {config_name} - {arch_cfg.get("desc", "")}'.strip())
    print('=' * 70)

    exp_start_time = time.time()

    L_layers = 2
    blocks_per_step, blocks_per_puzzle = _arch_compute_stats(H_cycles, L_cycles, L_layers, HALT_MAX_STEPS)

    base_blocks_per_step, _ = _arch_compute_stats(3, 4, L_layers, HALT_MAX_STEPS)
    total_blocks_budget = int(MAX_TRAIN_STEPS * base_blocks_per_step)
    max_steps = max(1, int(round(total_blocks_budget / max(1, blocks_per_step))))
    eval_every_steps = max(1, max_steps // 5)

    cfg_dict = {
        'arch': {
            'name': 'TRM',
            'loss': {'name': 'TRMLossHead', 'loss_type': 'stablemax_cross_entropy'},
            'halt_exploration_prob': 0.1,
            'halt_max_steps': HALT_MAX_STEPS,
            'halt_max_steps_eval': None,
            'disable_act': True,
            'H_cycles': H_cycles,
            'L_cycles': L_cycles,
            'H_layers': 0,
            'L_layers': L_layers,
            'hidden_size': 512,
            'num_heads': 8,
            'expansion': 4,
            'pos_encodings': 'rope',
            'forward_dtype': 'bfloat16',
            'puzzle_emb_ndim': 512,
            'puzzle_emb_len': 1,
            'mlp_t': False,
            'no_ACT_continue': True,
        },
        'data_paths': [DATASET_DIR],
        'data_paths_test': [],
        'global_batch_size': BATCH_SIZE,
        'max_steps': max_steps,
        'eval_every_steps': eval_every_steps,
        'min_eval_interval': MIN_EVAL_INTERVAL,
        'lr': 1e-4,
        'lr_min_ratio': 1.0,
        'lr_warmup_steps': 2000,
        'beta1': 0.9,
        'beta2': 0.95,
        'weight_decay': 0.1,
        'puzzle_emb_lr': 1e-2,
        'puzzle_emb_weight_decay': 0.1,
        'ema': True,
        'ema_rate': 0.999,
        'seed': 0,
        'project_name': 'TRM-A100-LH',
        'run_name': f'sudoku-{config_name}',
        'checkpoint_path': f'./checkpoints/{config_name}',
        'checkpoint_every_eval': False,
    }

    cfg = PretrainConfig(**cfg_dict)
    torch.manual_seed(cfg.seed)

    train_loader, train_meta = create_dataloader(cfg, 'train', epochs_per_iter=1)
    eval_loader, _ = create_dataloader(cfg, 'test', epochs_per_iter=1)

    ts = init_train_state(cfg, train_meta)

    use_wandb = bool(WANDB_API_KEY)
    if use_wandb:
        wandb.init(project=cfg.project_name, name=cfg.run_name, config={**cfg_dict, 'config_name': config_name})

    param_stats = count_parameters(ts.model)
    peak_memory = 0.0

    if use_wandb:
        wandb.log(
            {
                'model/total_params': param_stats['total'],
                'model/trainable_params': param_stats['trainable'],
                'model/trainable_ratio': param_stats['ratio'],
                'arch/H_cycles': H_cycles,
                'arch/L_cycles': L_cycles,
                'arch/halt_max_steps': HALT_MAX_STEPS,
                'arch/blocks_per_step': blocks_per_step,
                'arch/blocks_per_puzzle': blocks_per_puzzle,
                'compute/base_blocks_per_step': base_blocks_per_step,
                'compute/total_blocks_budget': total_blocks_budget,
                'compute/max_steps': max_steps,
                'compute/blocks_budget_ratio_to_base': (max_steps * blocks_per_step) / max(1, total_blocks_budget),
            },
            step=0,
        )

    print(f"📊 Params: Total={param_stats['total']:,}, Trainable={param_stats['trainable']:,} ({param_stats['ratio']:.2%})")
    print(f"📊 Arch: H_cycles={H_cycles}, L_cycles={L_cycles}, L_layers={L_layers}, halt_max_steps={HALT_MAX_STEPS}")
    print(f"📊 Approx blocks: {blocks_per_step} /step, {blocks_per_puzzle} /puzzle")
    print(f"📊 Compute budget: total_blocks_budget={total_blocks_budget:,}, max_steps={cfg.max_steps}, eval_every_steps={cfg.eval_every_steps}")

    ema = EMAHelper(cfg.ema_rate) if cfg.ema else None
    if ema:
        ema.register(ts.model)

    train_iter = iter(train_loader)
    last_eval_metrics = {}

    pbar = tqdm(total=cfg.max_steps)

    while ts.step < cfg.max_steps:
        try:
            _, batch, gbs = next(train_iter)
        except StopIteration:
            train_iter = iter(train_loader)
            continue

        ts.model.train()
        m = train_batch(cfg, ts, batch, gbs)
        if m:
            m['compute/blocks_per_step'] = float(blocks_per_step)
            m['compute/blocks_trained'] = float(ts.step * blocks_per_step)
            m['compute/blocks_budget'] = float(total_blocks_budget)

        if m and use_wandb:
            wandb.log(m, step=ts.step)

        if ema:
            ema.update(ts.model)

        pbar.update(ts.step - pbar.n)

        if (ts.step % cfg.eval_every_steps) == 0 and (ts.step // cfg.eval_every_steps) >= cfg.min_eval_interval:
            if ema:
                ts_eval = copy.deepcopy(ts)
                ts_eval.model = ema.ema_copy(ts.model)
            else:
                ts_eval = ts

            ts_eval.model.eval()
            em = evaluate(cfg, ts_eval, eval_loader, max_batches=None)
            if em:
                em['compute/blocks_per_step'] = float(blocks_per_step)
                em['compute/blocks_trained'] = float(ts.step * blocks_per_step)
                em['compute/blocks_budget'] = float(total_blocks_budget)

            last_eval_metrics = em

            if em:
                print(em)
                if use_wandb:
                    wandb.log(em, step=ts.step)

            if ema:
                del ts_eval

    pbar.close()

    total_time = time.time() - exp_start_time
    if torch.cuda.is_available():
        peak_memory = torch.cuda.max_memory_allocated() / 1e9

    summary = {
        'config': config_name,
        'H_cycles': H_cycles,
        'L_cycles': L_cycles,
        'halt_max_steps': HALT_MAX_STEPS,
        'blocks_per_step': blocks_per_step,
        'blocks_per_puzzle': blocks_per_puzzle,
        'base_blocks_per_step': base_blocks_per_step,
        'total_blocks_budget': total_blocks_budget,
        'max_steps': cfg.max_steps,
        'total_blocks_trained': int(ts.step * blocks_per_step),
        'blocks_budget_ratio_to_base': (ts.step * blocks_per_step) / max(1, total_blocks_budget),
        'trainable_params': param_stats['trainable'],
        'trainable_ratio': param_stats['ratio'],
        'peak_memory_gb': peak_memory,
        'total_time_s': total_time,
        'samples_per_sec': (ts.step * BATCH_SIZE) / max(total_time, 1),
        'final_accuracy': last_eval_metrics.get('eval/accuracy', 0),
        'final_exact_accuracy': last_eval_metrics.get('eval/exact_accuracy', 0),
    }

    if use_wandb:
        wandb.log(
            {
                'arch/peak_memory_gb': summary['peak_memory_gb'],
                'arch/samples_per_sec': summary['samples_per_sec'],
                'arch/final_exact_accuracy': summary['final_exact_accuracy'],
                'compute/total_blocks_trained': summary['total_blocks_trained'],
                'compute/blocks_budget_ratio_to_base': summary['blocks_budget_ratio_to_base'],
            },
            step=ts.step,
        )
        wandb.finish()

    print(f"\n📈 {config_name}: H={H_cycles}, L={L_cycles}, exact_acc={summary['final_exact_accuracy']:.2%}\n")

    return ts, summary

In [ ]:
! export DISABLE_COMPILE=1

In [ ]:
# ============================================================================
# Cell 13: Run Experiment(s) - Single or All Architectures
# ============================================================================
import pandas as pd

RUN_MODE = 'single'  # 'single' or 'all_ARCH'

if RUN_MODE == 'single':
    train_state, summary = launch_experiment(SELECTED_CONFIG)
    print(f"✅ {SELECTED_CONFIG}: {summary['final_exact_accuracy']:.2%} exact accuracy")

elif RUN_MODE == 'all_ARCH':
    all_summaries = []
    for name in sorted(ARCH_CONFIGS.keys()):
        _, summary = launch_experiment(name)
        all_summaries.append(summary)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    df = pd.DataFrame(all_summaries)

    print("\n" + "=" * 110)
    print("📊 ARCHITECTURE COMPARISON (H/L cycles)")
    print("=" * 110)
    cols = [
        'config',
        'H_cycles',
        'L_cycles',
        'halt_max_steps',
        'blocks_per_step',
        'blocks_per_puzzle',
        'trainable_params',
        'trainable_ratio',
        'peak_memory_gb',
        'samples_per_sec',
        'final_exact_accuracy',
    ]
    print(df[cols].round(6).to_string(index=False))
    print("=" * 110)

    df.to_csv('arch_results.csv', index=False)
    print("✅ Saved to arch_results.csv")

In [ ]:
# ============================================================================
# Cell 14: Architecture Comparison Visualization
# ============================================================================
import matplotlib.pyplot as plt

try:
    df
except NameError:
    df = pd.read_csv('arch_results.csv')

fig, axes = plt.subplots(1, 4, figsize=(18, 4))

# 1. H cycles
axes[0].bar(df['config'], df['H_cycles'])
axes[0].set_ylabel('H_cycles')
axes[0].set_title('H (outer cycles)')
axes[0].tick_params(axis='x', rotation=60)

# 2. L cycles
axes[1].bar(df['config'], df['L_cycles'])
axes[1].set_ylabel('L_cycles')
axes[1].set_title('L (inner cycles)')
axes[1].tick_params(axis='x', rotation=60)

# 3. Approx compute (blocks per puzzle)
axes[2].bar(df['config'], df['blocks_per_puzzle'])
axes[2].set_ylabel('Blocks / puzzle')
axes[2].set_title('Approx compute')
axes[2].tick_params(axis='x', rotation=60)

# 4. Exact accuracy
axes[3].bar(df['config'], df['final_exact_accuracy'])
axes[3].set_ylabel('Exact Accuracy')
axes[3].set_title('Performance')
axes[3].tick_params(axis='x', rotation=60)

plt.suptitle('TRM Architecture Comparison (H/L cycles)', fontweight='bold')
plt.tight_layout()
plt.savefig('arch_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved to arch_comparison.png")